#### 1. 测试 Redis 和 大模型 流式分析 

In [ ]:
# 初始化
import django_setup

In [ ]:
from apps.projects.models import Project, Task, TaskType, TaskStatus
# 模拟celery task 处获取 项目信息
current_project = Project.objects.get(project_name='测试项目1')
print(f"用于测试的项目: {current_project.project_name}")
print(f"项目包含的文件: {current_project.files.all()}")

In [ ]:
outline_task = Task.objects.get(stage__project=current_project, type=TaskType.DOCX_EXTRACTION_TASK)
raw_data = outline_task.docx_tiptap
print(f"raw_data: {raw_data}")

In [ ]:
from apps.projects.utils.redis_manager import RedisManager
redis_manager = RedisManager()

In [ ]:
from apps.projects.services._02_outline_analysis import DocxOutlineAnalyzerStep
import asyncio
analyzer = DocxOutlineAnalyzerStep()

stream_id = "1268930d-6aa3-44f1-89f8-f9c9631573b7"
# 执行流式分析 
asyncio.run(analyzer.process_streaming(raw_data, stream_id))

In [6]:
# 检查特定 stream_id 是否存在
result = redis_manager.get_stream_chunks(stream_id)

In [ ]:
print(result)

In [ ]:
result_status = redis_manager.get_stream_status(stream_id)
from pprint import pprint
pprint(result_status)

#### 2. 测试 Celery 和 Redis 和 大模型分析 

In [ ]:
# 初始化
import django_setup

In [ ]:
from apps.projects.models import Project, Task, TaskType, TaskStatus
# 模拟celery task 处获取 项目信息
current_project = Project.objects.get(project_name='测试项目1')
print(f"用于测试的项目: {current_project.project_name}")
print(f"项目包含的文件: {current_project.files.all()}")

In [ ]:
from apps.projects.utils.redis_manager import RedisManager
rm = RedisManager()
stream_key = rm.create_stream_key("b3611410-a723-4c95-8fc3-aad7bf637b33")
print(stream_key)  # 应该是 "model_stream:b3611410-a723-4c95-8fc3-aad7bf637b33"
print(rm.redis_client.exists(stream_key))  # 是否存在该键
print(rm.redis_client.llen(stream_key))  # 该键对应的列表长度

In [ ]:
outline_task = Task.objects.get(stage__project=current_project, type=TaskType.OUTLINE_ANALYSIS_TASK)
outline_task.status = TaskStatus.ACTIVE
outline_task.save()

In [ ]:
from apps.projects.utils.redis_manager import RedisManager
redis_manager = RedisManager()
from apps.projects.tasks import process_outline_analysis_streaming

In [ ]:
project_id = current_project.id
celery_task = process_outline_analysis_streaming.delay(project_id)
celery_task_id = celery_task.id
print(f"启动大纲分析任务: project_id={project_id}")
print(f"启动大纲分析任务: celery_task_id={celery_task_id}")



In [ ]:

result_status = redis_manager.get_stream_status(celery_task_id)
from pprint import pprint
pprint(result_status)